In [ ]:
# --- Cell 1: Tải các thư viện ---
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Input
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

# Thư viện cho các mô hình mới (nếu chưa cài sẽ lỗi — cài trước khi chạy)
try:
    import xgboost as xgb
except Exception as e:
    xgb = None
    print("Không import được xgboost:", e)

try:
    import catboost as cb
except Exception as e:
    cb = None
    print("Không import được catboost:", e)

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
print(f"Python: {pd.__version__}, NumPy: {np.__version__}, TensorFlow: {tf.__version__}")
if xgb is not None:
    print(f"XGBoost version: {xgb.__version__}")
if cb is not None:
    try:
        print(f"CatBoost version: {cb.__version__}")
    except Exception:
        pass


In [ ]:
# --- Cell 2: Tải dữ liệu đã xử lý ---
file_name = 'kpi_processed.csv'

try:
    df = pd.read_csv(file_name, index_col=0, parse_dates=True)
    # Đảm bảo tên index là 'timestamp' để nhất quán
    if df.index.name is None:
        df.index.name = 'timestamp'
    print(f"Tải file '{file_name}' thành công.")
    print(f"Dữ liệu có {len(df)} hàng, kéo dài từ {df.index.min()} đến {df.index.max()}.")
except Exception as e:
    df = None
    print(f"LỖI: Không thể tải file '{file_name}'.")
    print(e)


In [ ]:
# --- Cell 3: Xử lý dữ liệu (Đảm bảo chuỗi thời gian đầy đủ) ---
if df is not None:
    # Thay đổi theo file của bạn nếu khác
    FEATURE_COLS = ['ps_traffic_mb', 'avg_rrc_connected_user', 'prb_dl_used', 'prb_dl_available_total', 'prb_utilization']
    # Kiểm tra tồn tại cột
    missing = [c for c in FEATURE_COLS + ['cell_name'] if c not in df.columns]
    if missing:
        raise ValueError(f"Thiếu cột trong DataFrame: {missing}")

    N_FEATURES = len(FEATURE_COLS)
    print(f"Dự đoán {N_FEATURES} features: {FEATURE_COLS}")

    print("Đang xử lý: Đảm bảo dữ liệu 15 phút đầy đủ cho mỗi cell...")
    all_cells = df['cell_name'].unique()
    full_time_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='15min')

    # Reset index để có cột timestamp
    df_data_only = df[FEATURE_COLS + ['cell_name']].reset_index()
    # Đảm bảo tên cột thời gian là 'timestamp'
    if 'timestamp' not in df_data_only.columns:
        # nếu index name khác, rename
        df_data_only = df_data_only.rename(columns={df_data_only.columns[0]: 'timestamp'})

    # groupby trung bình nếu có trùng timestamp-cell
    df_grouped_unique = df_data_only.groupby(['cell_name', 'timestamp']).mean()

    multi_index = pd.MultiIndex.from_product([all_cells, full_time_index], names=['cell_name', 'timestamp'])
    df_full = df_grouped_unique.reindex(multi_index, fill_value=0).reset_index().set_index('timestamp')

    print(f"Dữ liệu đã điền đầy đủ (sau reindex) có {len(df_full)} hàng.")
else:
    print("Lỗi: DataFrame 'df' không tồn tại. Vui lòng chạy lại Cell 2.")


In [ ]:
# --- Cell 4: Cấu hình và Tách dữ liệu (Train/Val/Test) ---
TIMESTEPS_PER_HOUR = 4
TIMESTEPS_PER_DAY = 24 * TIMESTEPS_PER_HOUR  # 96

INPUT_DAYS = 2
INPUT_STEPS = INPUT_DAYS * TIMESTEPS_PER_DAY  # 192

OUTPUT_DAYS = 1
OUTPUT_STEPS = OUTPUT_DAYS * TIMESTEPS_PER_DAY  # 96

print(f"Cấu hình mô hình: Input {INPUT_STEPS} bước (2 ngày), Output {OUTPUT_STEPS} bước (1 ngày).")

if 'df_full' in locals():
    total_duration = df_full.index.max() - df_full.index.min()
    train_end_time = df_full.index.min() + total_duration * 0.7
    val_end_time = df_full.index.min() + total_duration * 0.9

    train_df = df_full[df_full.index < train_end_time].copy()
    val_df = df_full[(df_full.index >= train_end_time) & (df_full.index < val_end_time)].copy()
    test_df = df_full[df_full.index >= val_end_time].copy()

    print(f"Kích thước tập Train: {train_df.shape}")
    print(f"Kích thước tập Val:   {val_df.shape}")
    print(f"Kích thước tập Test:  {test_df.shape}")
else:
    print("Lỗi: DataFrame 'df_full' không tồn tại.")


In [ ]:
# --- Cell 5: Chuẩn hóa (Scaling) Dữ liệu ---
if 'train_df' in locals():
    # Xử lý NaN/Inf
    train_df = train_df.replace([np.inf, -np.inf], 0).fillna(0)
    val_df = val_df.replace([np.inf, -np.inf], 0).fillna(0)
    test_df = test_df.replace([np.inf, -np.inf], 0).fillna(0)

    scaler = MinMaxScaler()
    scaler.fit(train_df[FEATURE_COLS])

    # Transform và tạo lại DataFrame
    scaled_train_df = pd.DataFrame(scaler.transform(train_df[FEATURE_COLS]), columns=FEATURE_COLS, index=train_df.index)
    scaled_train_df['cell_name'] = train_df['cell_name'].values

    scaled_val_df = pd.DataFrame(scaler.transform(val_df[FEATURE_COLS]), columns=FEATURE_COLS, index=val_df.index)
    scaled_val_df['cell_name'] = val_df['cell_name'].values

    scaled_test_df = pd.DataFrame(scaler.transform(test_df[FEATURE_COLS]), columns=FEATURE_COLS, index=test_df.index)
    scaled_test_df['cell_name'] = test_df['cell_name'].values

    print("Hoàn tất scaling dữ liệu.")
else:
    print("Lỗi: Dữ liệu train không tồn tại.")


In [ ]:
# --- Cell 6: Hàm tạo cửa sổ (Windowing) ---
def create_windows(data_df, input_steps, output_steps, feature_cols):
    X, y = [], []
    grouped = data_df.groupby('cell_name')
    print(f"Bắt đầu tạo cửa sổ cho {len(grouped)} cell...")

    total_windows = 0
    for cell_id, cell_data in grouped:
        cell_features = cell_data[feature_cols].values
        total_samples = len(cell_features)
        total_window_len = input_steps + output_steps

        if total_samples < total_window_len:
            continue

        for i in range(total_samples - total_window_len + 1):
            input_end = i + input_steps
            output_end = input_end + output_steps

            X.append(cell_features[i:input_end, :])
            y.append(cell_features[input_end:output_end, :])
            total_windows += 1

    print(f"Hoàn tất tạo cửa sổ. Đã tạo {total_windows} mẫu.")
    return np.array(X), np.array(y)


In [ ]:
# --- Cell 7: Áp dụng hàm Windowing cho LSTM/Transformer ---
if 'scaled_train_df' in locals():
    print("--- Đang tạo mẫu cho mô hình Deep Learning ---")
    X_train, y_train = create_windows(scaled_train_df, INPUT_STEPS, OUTPUT_STEPS, FEATURE_COLS)
    X_val, y_val = create_windows(scaled_val_df, INPUT_STEPS, OUTPUT_STEPS, FEATURE_COLS)
    X_test, y_test = create_windows(scaled_test_df, INPUT_STEPS, OUTPUT_STEPS, FEATURE_COLS)

    print("\n--- Kích thước dữ liệu (Shape) ---")
    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"X_val shape:   {X_val.shape}")
    print(f"y_val shape:   {y_val.shape}")
    print(f"X_test shape:  {X_test.shape}")
    print(f"y_test shape:  {y_test.shape}")
else:
    print("Lỗi: Dữ liệu đã scale không tồn tại.")


In [ ]:
# --- Cell 8: Chuẩn bị dữ liệu cho XGBoost/CatBoost ---
if 'X_train' in locals() and X_train.size > 0:
    # Làm phẳng input: (samples, input_steps, n_features) -> (samples, input_steps * n_features)
    X_train_tree = X_train.reshape(X_train.shape[0], -1)
    X_val_tree = X_val.reshape(X_val.shape[0], -1) if X_val.size > 0 else np.empty((0, X_train_tree.shape[1]))
    X_test_tree = X_test.reshape(X_test.shape[0], -1) if X_test.size > 0 else np.empty((0, X_train_tree.shape[1]))

    # Làm phẳng output: (samples, output_steps, n_features) -> (samples, output_steps * n_features)
    y_train_tree = y_train.reshape(y_train.shape[0], -1)
    y_val_tree = y_val.reshape(y_val.shape[0], -1) if y_val.size > 0 else np.empty((0, y_train_tree.shape[1]))
    y_test_tree = y_test.reshape(y_test.shape[0], -1) if y_test.size > 0 else np.empty((0, y_train_tree.shape[1]))

    print("--- Dữ liệu cho mô hình cây (XGBoost, CatBoost) ---")
    print(f"X_train_tree shape: {X_train_tree.shape}")
    print(f"y_train_tree shape: {y_train_tree.shape}")
else:
    print("Lỗi: Dữ liệu X_train chưa được tạo hoặc rỗng.")


In [ ]:
# --- Cell 9: Xây dựng và Compile mô hình LSTM ---
def build_lstm_model(input_steps, n_features, output_steps):
    keras.backend.clear_session()
    model = Sequential([
        Input(shape=(input_steps, n_features)),
        LSTM(100, activation='tanh'),
        RepeatVector(output_steps),
        LSTM(100, activation='tanh', return_sequences=True),
        TimeDistributed(Dense(n_features))
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

if 'X_train' in locals() and X_train.size > 0:
    model_lstm = build_lstm_model(INPUT_STEPS, N_FEATURES, OUTPUT_STEPS)
    print("--- LSTM Architecture ---")
    model_lstm.summary()
else:
    print("Lỗi: Không có dữ liệu training (X_train) để xây dựng mô hình.")


In [ ]:
# --- Cell 10: Huấn luyện mô hình LSTM ---
if 'model_lstm' in locals():
    print("Bắt đầu huấn luyện (training) mô hình LSTM...")
    callbacks = [
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    ]
    history_lstm = model_lstm.fit(
        X_train, y_train,
        epochs=20,
        batch_size=64,
        validation_data=(X_val, y_val),
        callbacks=callbacks,
        verbose=1
    )
    print("Hoàn tất huấn luyện LSTM!")
else:
    print("Lỗi: Mô hình LSTM chưa được định nghĩa.")


In [ ]:
# --- Cell 11: Xây dựng và Compile mô hình Transformer ---
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.0):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

def build_transformer_model(input_shape, output_steps, output_features, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0.0, mlp_dropout=0.0):
    keras.backend.clear_session()
    inputs = keras.Input(shape=input_shape)
    x = inputs
    x = layers.Dense(head_size)(x)
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
    # Global pooling across time dimension
    x = layers.GlobalAveragePooling1D()(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    x = layers.Dense(output_steps * output_features)(x)
    outputs = layers.Reshape((output_steps, output_features))(x)
    model = keras.Model(inputs, outputs)
    optimizer = keras.optimizers.Adam(learning_rate=1e-3)
    model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])
    return model

if 'X_train' in locals() and X_train.size > 0:
    model_transformer = build_transformer_model(
        input_shape=(INPUT_STEPS, N_FEATURES),
        output_steps=OUTPUT_STEPS, output_features=N_FEATURES,
        head_size=64, num_heads=4, ff_dim=128, num_transformer_blocks=2,
        mlp_units=[128], dropout=0.1, mlp_dropout=0.1
    )
    print("--- Transformer Architecture ---")
    model_transformer.summary()
else:
    print("Lỗi: Thiếu dữ liệu X_train.")


In [ ]:
# --- Cell 12: Huấn luyện mô hình Transformer ---
if 'model_transformer' in locals():
    print("Bắt đầu huấn luyện Transformer...")
    callbacks = [
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6)
    ]
    history_transformer = model_transformer.fit(
        X_train, y_train,
        batch_size=128,
        epochs=50,
        validation_data=(X_val, y_val),
        callbacks=callbacks,
        verbose=1
    )
    print("Hoàn tất huấn luyện Transformer!")
else:
    print("Lỗi: Mô hình Transformer chưa được định nghĩa.")


In [ ]:
# --- Cell 13: Huấn luyện mô hình XGBoost ---
if xgb is None:
    print("XGBoost chưa được cài/import — bỏ qua phần XGBoost.")
elif 'X_train_tree' in locals() and X_train_tree.size > 0:
    print("Bắt đầu huấn luyện XGBoost...")

    model_xgb = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        tree_method='hist'  # đổi thành 'gpu_hist' nếu có GPU và muốn dùng GPU
    )

    eval_set = [(X_val_tree, y_val_tree)] if X_val_tree.size > 0 else None
    if eval_set is not None:
        model_xgb.fit(
            X_train_tree, y_train_tree,
            eval_set=eval_set,
            early_stopping_rounds=50,
            verbose=100
        )
    else:
        model_xgb.fit(X_train_tree, y_train_tree, verbose=1)

    print("Hoàn tất huấn luyện XGBoost!")
else:
    print("Lỗi: Dữ liệu X_train_tree chưa được tạo hoặc rỗng.")


In [ ]:
# --- Cell 14: Huấn luyện mô hình CatBoost ---
if cb is None:
    print("CatBoost chưa được cài/import — bỏ qua phần CatBoost.")
elif 'X_train_tree' in locals() and X_train_tree.size > 0:
    print("Bắt đầu huấn luyện CatBoost...")

    model_catboost = cb.CatBoostRegressor(
        iterations=1000,
        learning_rate=0.05,
        depth=6,
        loss_function='MultiRMSE',
        eval_metric='MultiRMSE',
        random_seed=42,
        verbose=100,
        task_type="CPU"  # đổi "GPU" nếu có GPU
    )

    eval_set = (X_val_tree, y_val_tree) if X_val_tree.size > 0 else None
    if eval_set is not None:
        model_catboost.fit(
            X_train_tree, y_train_tree,
            eval_set=eval_set,
            early_stopping_rounds=50,
            verbose=100
        )
    else:
        model_catboost.fit(X_train_tree, y_train_tree, verbose=100)

    print("Hoàn tất huấn luyện CatBoost!")
else:
    print("Lỗi: Dữ liệu X_train_tree chưa được tạo hoặc rỗng.")


In [ ]:
# --- Cell 15: So sánh Loss Curves (LSTM vs Transformer) ---
if 'history_lstm' in locals() and 'history_transformer' in locals():
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7))

    # So sánh MSE Loss
    ax1.plot(history_lstm.history['val_loss'], label='LSTM Validation Loss')
    ax1.plot(history_transformer.history['val_loss'], label='Transformer Validation Loss')
    ax1.set_title('So sánh Validation Loss (MSE)')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Mean Squared Error (MSE)')
    ax1.legend()

    # So sánh MAE
    ax2.plot(history_lstm.history.get('val_mae', []), label='LSTM Validation MAE')
    ax2.plot(history_transformer.history.get('val_mae', []), label='Transformer Validation MAE')
    ax2.set_title('So sánh Validation MAE')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Mean Absolute Error (MAE)')
    ax2.legend()

    plt.suptitle('So sánh quá trình huấn luyện LSTM và Transformer', fontsize=16)
    plt.show()
else:
    print("Lỗi: Chưa có history của LSTM hoặc Transformer.")


In [ ]:
# --- Cell 16: So sánh Loss Curves (XGBoost vs CatBoost) ---
if ('model_xgb' in locals() and xgb is not None) or ('model_catboost' in locals() and cb is not None):
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(1, 1, figsize=(12, 7))

    if 'model_xgb' in locals() and xgb is not None:
        try:
            xgb_results = model_xgb.evals_result()
            # có thể có key 'validation_0' with 'rmse'
            for k, v in xgb_results.items():
                # lấy metric list nếu có
                for metric_name, metric_vals in v.items():
                    ax.plot(metric_vals, label=f'XGBoost {k}-{metric_name}')
        except Exception as e:
            print("Không lấy được evals_result từ XGBoost:", e)

    if 'model_catboost' in locals() and cb is not None:
        try:
            catboost_results = model_catboost.get_evals_result()
            # catboost_results thường có key 'validation' -> dict of metrics
            for metric_name, metric_vals in catboost_results.get('validation', {}).items():
                ax.plot(metric_vals, label=f'CatBoost validation-{metric_name}')
        except Exception as e:
            print("Không lấy được get_evals_result từ CatBoost:", e)

    ax.set_title('So sánh Validation Loss (XGBoost vs CatBoost)')
    ax.set_xlabel('Boosting Round / Epoch')
    ax.set_ylabel('Metric')
    ax.legend()
    ax.grid(True)
    plt.show()
else:
    print("Lỗi: Chưa có kết quả huấn luyện của XGBoost hoặc CatBoost.")


In [ ]:
# --- Cell 17: Đánh giá trên tập Test ---
if 'X_test' in locals() and X_test.size > 0:
    results = {}

    # 1. Đánh giá LSTM
    if 'model_lstm' in locals():
        loss, mae = model_lstm.evaluate(X_test, y_test, verbose=0)
        results['LSTM'] = {'MSE': float(loss), 'MAE': float(mae)}
        print(f"LSTM Test Results: MSE={loss:.6f}, MAE={mae:.6f}")

    # 2. Đánh giá Transformer
    if 'model_transformer' in locals():
        loss, mae = model_transformer.evaluate(X_test, y_test, verbose=0)
        results['Transformer'] = {'MSE': float(loss), 'MAE': float(mae)}
        print(f"Transformer Test Results: MSE={loss:.6f}, MAE={mae:.6f}")

    # 3. Đánh giá XGBoost
    if 'model_xgb' in locals() and xgb is not None:
        y_pred_xgb_flat = model_xgb.predict(X_test_tree)
        mse = mean_squared_error(y_test_tree, y_pred_xgb_flat)
        mae = mean_absolute_error(y_test_tree, y_pred_xgb_flat)
        results['XGBoost'] = {'MSE': float(mse), 'MAE': float(mae)}
        print(f"XGBoost Test Results: MSE={mse:.6f}, MAE={mae:.6f}")

    # 4. Đánh giá CatBoost
    if 'model_catboost' in locals() and cb is not None:
        y_pred_catboost_flat = model_catboost.predict(X_test_tree)
        mse = mean_squared_error(y_test_tree, y_pred_catboost_flat)
        mae = mean_absolute_error(y_test_tree, y_pred_catboost_flat)
        results['CatBoost'] = {'MSE': float(mse), 'MAE': float(mae)}
        print(f"CatBoost Test Results: MSE={mse:.6f}, MAE={mae:.6f}")

    # In bảng kết quả
    results_df = pd.DataFrame(results).T
    print("\n--- Bảng tổng kết kết quả trên tập Test ---")
    print(results_df)
else:
    print("Lỗi: Thiếu dữ liệu test hoặc X_test rỗng.")
